In [1]:
%pip install requests pandas google-cloud-bigquery apache-airflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Ale\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [11]:
import os
import requests
import pandas as pd
from google.cloud import bigquery
from datetime import datetime, timedelta

In [17]:
# Constants
CLOUDINARY_API_URL = "https://api.cloudinary.com/v1_1/dqyf4lrlo/resources/search"
MONGODB_API_URL = "https://bahela-admin.vercel.app/api/sessions"
CLOUDINARY_AUTH = ("214285446753493", "_Lwj_-7ml1KqQAe4Oadpaxoqhf4")
START_DATE = (datetime.utcnow() - timedelta(days=10)).strftime("%Y-%m-%d")
END_DATE = (datetime.utcnow() - timedelta(days=10)).strftime("%Y-%m-%d")

In [18]:
def fetch_cloudinary_data(start_date, end_date):
    all_data = []
    next_cursor = None
    while True:
        # Modify the expression to include the date range
        params = {
            "expression": f"created_at:[{start_date} TO {end_date}]",
            "max_results": 50,
        }
        if next_cursor:
        params["next_cursor"] = next_cursor
        response = requests.get(CLOUDINARY_API_URL, auth=CLOUDINARY_AUTH, params=params)
        if response.status_code == 200:
            data = response.json()
            all_data.extend(data.get("resources", []))
            next_cursor = data.get("next_cursor")
            if not next_cursor:  # Break if no more pages
                break
        else:
            print(f"Failed to fetch Cloudinary data: {response.text}")
            break
    return pd.DataFrame(all_data)

# Fetch Cloudinary data for the last three months
cloudinary_data = fetch_cloudinary_data(START_DATE, END_DATE)
cloudinary_data.head()

,asset_id,public_id,asset_folder,filename,display_name,format,version,resource_type,type,created_at,...,aspect_ratio,pixels,url,secure_url,status,access_mode,access_control,etag,created_by,uploaded_by
0,aad9cf11f4c666f64b8bc8785fafdcc1,jhfkv1-7,,jhfkv1-7,jhfkv1-7,png,1731688799,image,upload,2024-11-15T16:39:59+00:00,...,0.66543,779040,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,d711e1ed24db01df363c9c856d9d788d,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
1,6bec97b2f02a1a1ed8550297dde61888,jhfkv1-6,,jhfkv1-6,jhfkv1-6,png,1731688798,image,upload,2024-11-15T16:39:58+00:00,...,0.33179,388438,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,490d5aa646be2386bcfb87af14e7843a,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
2,6e787cb9c245d5dcf5541b6dd0d7794d,jhfkv1-5,,jhfkv1-5,image-6,png,1731688696,image,upload,2024-11-15T16:38:16+00:00,...,1.00000,409600,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,8ff5d3c73ba3a195a789cdce61091cd3,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
3,bc5f35109d7079a0d4315b88155b42d2,jhfkv1-4,,jhfkv1-4,image-5,png,1731688694,image,upload,2024-11-15T16:38:14+00:00,...,1.00000,409600,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,d10c8063df339b73b16d27547c610426,{'access_key': '214285446753493'},{'access_key': '214285446753493'}
4,e2a5688d633e3565200297a6e2029152,jhfkv1-3,,jhfkv1-3,image-4,png,1731688693,image,upload,2024-11-15T16:38:13+00:00,...,1.00000,409600,http://res.cloudinary.com/dqyf4lrlo/image/uplo...,https://res.cloudinary.com/dqyf4lrlo/image/upl...,active,public,None,77fbe19e165520da891f48396d48613e,{'access_key': '214285446753493'},{'access_key': '214285446753493'}


In [19]:
def fetch_mongodb_data(start_date, end_date):
    # Parameters with incremented end_date
    params = {"startDate": start_date, "endDate": end_date}
    response = requests.get(MONGODB_API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data.get("appStates", []))
    else:
        print(f"Failed to fetch MongoDB data: {response.text}")
        return pd.DataFrame()

# Fetch MongoDB data for yesterday only
mongodb_data = fetch_mongodb_data(START_DATE, END_DATE)
mongodb_data.head() 

,progress,options,payment,result,_id,id,page,__v
0,"{'appStartTime': '2024-11-15T00:07:03.881Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 35000, 'orderId': 'order-jhfkv1-173...",{'capturedImageUrls': ['blob:file:///cd3f664b-...,673779614d805da12198cb95,jhfkv1,Result,0
1,"{'appStartTime': '2024-11-15T00:07:03.881Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 35000, 'orderId': 'order-6hql8y-173...",{'capturedImageUrls': ['blob:file:///03a9cc73-...,673767f8d214213c25424648,6hql8y,Result,0
2,"{'appStartTime': '2024-11-15T00:07:03.881Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 35000, 'orderId': 'order-m8o8ei-173...",{'capturedImageUrls': ['blob:file:///e5eec645-...,673764ae531a800029116294,m8o8ei,Result,0
3,"{'appStartTime': '2024-11-15T00:07:03.881Z', '...","{'lang': 'id', 'filterFn': 'transformToMentari...","{'amount': 35000, 'orderId': 'order-e3rco1-173...",{'capturedImageUrls': ['blob:file:///e6e4976f-...,673761c0f7f84d6c6ea332a5,e3rco1,Result,0
4,"{'appStartTime': '2024-11-15T00:07:03.881Z', '...","{'lang': 'id', 'filterFn': 'transformToJelaga'...","{'amount': 35000, 'orderId': 'order-gdamqs-173...",{'capturedImageUrls': ['blob:file:///023a2522-...,67375c3ba6397528301604d2,gdamqs,Result,0


In [32]:
# Section 1: Create joined_data
def create_joined_data(cloudinary_df, mongodb_df):

    # Filter MongoDB data
    filtered_mongodb = mongodb_df[
        (mongodb_df["page"] == "Result") &
        (mongodb_df["payment"].apply(lambda x: x.get("paid", False)))
    ]

    print(f"Filtered MongoDB size: {filtered_mongodb.shape[0]}")
    
    # Flatten progress, including start and end time
    def flatten_progress(progress):
        return {k: v for k, v in progress.items() if "error" not in k}
    
    filtered_mongodb["progress"] = filtered_mongodb["progress"].apply(flatten_progress)
    filtered_mongodb = filtered_mongodb.assign(
        app_start_time=filtered_mongodb["progress"].apply(lambda x: x.get("appStartTime")),
        start_time=filtered_mongodb["progress"].apply(lambda x: x.get("startTime")),
        end_time=filtered_mongodb["progress"].apply(lambda x: x.get("endTime"))
    )
    
    # Ensure start_time and end_time are datetime objects
    filtered_mongodb["start_time"] = pd.to_datetime(filtered_mongodb["start_time"], errors='coerce')
    filtered_mongodb["end_time"] = pd.to_datetime(filtered_mongodb["end_time"], errors='coerce')

    # Calculate session duration (end_time - start_time)
    filtered_mongodb["duration"] = (filtered_mongodb["end_time"] - filtered_mongodb["start_time"]).dt.total_seconds()

    # Flatten options and uniform column names
    filtered_mongodb = filtered_mongodb.assign(
        lang=filtered_mongodb["options"].apply(lambda x: x.get("lang")),
        filter_fn=filtered_mongodb["options"].apply(lambda x: x.get("filterFn")),
        copies=filtered_mongodb["options"].apply(lambda x: x.get("copies")),
        use_retry=filtered_mongodb["options"].apply(lambda x: x.get("useRetryMode")),  # Added retryCount
        retry_count=filtered_mongodb["options"].apply(lambda x: x.get("retryCount", 0) if x.get("useRetryMode", False) else 0)
    )
    
    # Flatten payment details and add discount and expiry_time
    filtered_mongodb = filtered_mongodb.assign(
        amount=filtered_mongodb["payment"].apply(lambda x: x.get("amount")),
        order_id=filtered_mongodb["payment"].apply(lambda x: x.get("orderId")),
        discount=filtered_mongodb["payment"].apply(lambda x: x.get("discount", 0)),  # Default to 0 if not found
        expiry_time=filtered_mongodb["payment"].apply(lambda x: x.get("expiryTime"))  # Assuming it's a timestamp or datetime string
    )
    
    # Extract and link uploadedImageUrls with Cloudinary data
    def map_uploaded_images(row):
        # Get the uploadedImageUrls or fallback to an empty list
        uploaded_urls = row["result"].get("uploadedImageUrls", [])

        print(uploaded_urls)

        # Filter Cloudinary data for matching public_ids and ensure public_id includes MongoDB id
        matched_images = cloudinary_df[cloudinary_df['public_id'].str.contains(row['id'])]

        # Return required fields as a list of dictionaries
        result = matched_images[[
            "asset_id", "public_id", "resource_type", "secure_url", "bytes", 
            "width", "height", "aspect_ratio", "pixels", "created_at"
        ]].to_dict("records")

        print(result)

        return result

    filtered_mongodb["uploaded_images"] = filtered_mongodb.apply(map_uploaded_images, axis=1)
    

    # Calculate bytes and pixels summary for each session
    def calculate_summary(row):
        # Extract public_ids from uploaded_images
        uploaded_ids = [img["public_id"] for img in row["uploaded_images"]]
        
        # Filter Cloudinary data for matching public_ids
        related_resources = cloudinary_df[cloudinary_df["public_id"].isin(uploaded_ids)]
        
        # Calculate totals and averages
        total_bytes = related_resources["bytes"].sum()
        avg_bytes = related_resources["bytes"].mean() if not related_resources.empty else 0
        total_pixels = related_resources["pixels"].sum()
        avg_pixels = related_resources["pixels"].mean() if not related_resources.empty else 0
        
        # Return a pandas Series with calculated values
        return pd.Series({
            "total_bytes": total_bytes,
            "avg_bytes": avg_bytes,
            "total_pixels": total_pixels,
            "avg_pixels": avg_pixels
        })

    bytes_pixels_summary = filtered_mongodb.apply(calculate_summary, axis=1)
    filtered_mongodb = pd.concat([filtered_mongodb, bytes_pixels_summary], axis=1)
    
    # Create final joined_data
    joined_data = filtered_mongodb[[
        "_id", "id", "app_start_time", "start_time", "end_time", "duration", "lang", "copies",
        "filter_fn", "amount", "order_id", "discount", "expiry_time", "use_retry", "retry_count", 
        "uploaded_images", "total_bytes", "avg_bytes", "total_pixels", "avg_pixels"
    ]].rename(columns={
        "_id": "id", 
        "id": "session_id", 
        "amount": "transaction_amount",
        "retry_count": "retry_attempts"
    })
    
    return joined_data

# Call the function to create joined_data
joined_data = create_joined_data(cloudinary_data, mongodb_data)
joined_data.head()  # Preview the result

Filtered MongoDB size: 25
['https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688689/jhfkv1-0.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688690/jhfkv1-1.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688692/jhfkv1-2.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688693/jhfkv1-3.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688694/jhfkv1-4.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688696/jhfkv1-5.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688798/jhfkv1-6.png', 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688799/jhfkv1-7.png']
[{'asset_id': 'aad9cf11f4c666f64b8bc8785fafdcc1', 'public_id': 'jhfkv1-7', 'resource_type': 'image', 'secure_url': 'https://res.cloudinary.com/dqyf4lrlo/image/upload/v1731688799/jhfkv1-7.png', 'bytes': 1350259, 'width': 720, 'height': 1082, 'aspect_ratio': 0.66543, 'pixels': 779040, 'created_at': '2024-11-15T16:39:59+00:00'}, {'asset_id': '6bec97b

C:\Users\Ale\AppData\Local\Temp\ipykernel_19228\2904914727.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_mongodb["progress"] = filtered_mongodb["progress"].apply(flatten_progress)


,id,session_id,app_start_time,start_time,end_time,duration,lang,copies,filter_fn,transaction_amount,order_id,discount,expiry_time,use_retry,retry_attempts,uploaded_images,total_bytes,avg_bytes,total_pixels,avg_pixels
0,673779614d805da12198cb95,jhfkv1,2024-11-15T00:07:03.881Z,2024-11-15 16:34:19.855000+00:00,2024-11-15 16:39:59.941000+00:00,340.086,id,1,transformToMentari,35000,order-jhfkv1-1731688500519,0,2024-11-15T23:37:01.000Z,True,3,[{'asset_id': 'aad9cf11f4c666f64b8bc8785fafdcc...,6811729.0,851466.125,3625078.0,453134.75
1,673767f8d214213c25424648,6hql8y,2024-11-15T00:07:03.881Z,2024-11-15 15:21:46.075000+00:00,2024-11-15 15:25:43.236000+00:00,237.161,id,1,transformToMentari,35000,order-6hql8y-1731684131757,0,2024-11-15T22:24:12.000Z,True,3,[{'asset_id': 'e2b8610cb7e5e1828b8a1e669c2f7b1...,6922417.0,865302.125,3625078.0,453134.75
2,673764ae531a800029116294,m8o8ei,2024-11-15T00:07:03.881Z,2024-11-15 15:06:24.957000+00:00,2024-11-15 15:11:40.850000+00:00,315.893,id,1,transformToMentari,35000,order-m8o8ei-1731683227256,0,2024-11-15T22:09:08.000Z,True,1,[{'asset_id': '677ca00a18cd57202df0f8810851dcd...,6647789.0,830973.625,3625078.0,453134.75
3,673761c0f7f84d6c6ea332a5,e3rco1,2024-11-15T00:07:03.881Z,2024-11-15 14:54:42.354000+00:00,2024-11-15 14:59:11.308000+00:00,268.954,id,1,transformToMentari,35000,order-e3rco1-1731682500940,0,2024-11-15T21:57:01.000Z,True,1,[{'asset_id': '28b3015e98521d1edd9aaeef88198b4...,6893376.0,861672.000,3625078.0,453134.75
4,67375c3ba6397528301604d2,gdamqs,2024-11-15T00:07:03.881Z,2024-11-15 14:31:49.048000+00:00,2024-11-15 14:35:36.542000+00:00,227.494,id,1,transformToJelaga,35000,order-gdamqs-1731681131826,0,2024-11-15T21:34:12.000Z,True,2,[{'asset_id': '8a191c0e09aa78d196272dc95e2e519...,3360739.0,420092.375,3625078.0,453134.75


In [22]:
# Section 2: Create fact_sessions
def create_fact_sessions(joined_data):
    fact_sessions = joined_data[[
        "session_id", "start_time", "duration", "transaction_amount", "discount", "retry_attempts", "copies",
        "total_bytes", "avg_bytes", "total_pixels", "avg_pixels"
    ]]
    
    return fact_sessions

# Create fact_sessions from joined_data
fact_sessions = create_fact_sessions(joined_data)
fact_sessions.head()  # Preview the result

,session_id,start_time,duration,transaction_amount,discount,retry_attempts,copies,total_bytes,avg_bytes,total_pixels,avg_pixels
0,jhfkv1,2024-11-15 16:34:19.855000+00:00,340.086,35000,0,3,1,6811729.0,851466.125,3625078.0,453134.75
1,6hql8y,2024-11-15 15:21:46.075000+00:00,237.161,35000,0,3,1,6922417.0,865302.125,3625078.0,453134.75
2,m8o8ei,2024-11-15 15:06:24.957000+00:00,315.893,35000,0,1,1,6647789.0,830973.625,3625078.0,453134.75
3,e3rco1,2024-11-15 14:54:42.354000+00:00,268.954,35000,0,1,1,6893376.0,861672.000,3625078.0,453134.75
4,gdamqs,2024-11-15 14:31:49.048000+00:00,227.494,35000,0,2,1,3360739.0,420092.375,3625078.0,453134.75


In [23]:
# Section 3: Create dim_sessions
def create_dim_sessions(joined_data):
    dim_sessions = joined_data[[
        "session_id", "app_start_time", "start_time", "end_time",
        "use_retry", "lang", "filter_fn", "order_id", "expiry_time", "uploaded_images"
    ]]
    
    # Convert 'expiry_time' and 'app_start_time' to datetime if needed
    dim_sessions["expiry_time"] = pd.to_datetime(dim_sessions["expiry_time"], errors='coerce')
    dim_sessions["app_start_time"] = pd.to_datetime(dim_sessions["app_start_time"], errors='coerce')

    return dim_sessions

# Create dim_sessions from joined_data
dim_sessions = create_dim_sessions(joined_data)
dim_sessions.head()  # Preview the result


C:\Users\Ale\AppData\Local\Temp\ipykernel_19228\2771637358.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_sessions["expiry_time"] = pd.to_datetime(dim_sessions["expiry_time"], errors='coerce')
C:\Users\Ale\AppData\Local\Temp\ipykernel_19228\2771637358.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_sessions["app_start_time"] = pd.to_datetime(dim_sessions["app_start_time"], errors='coerce')


,session_id,app_start_time,start_time,end_time,use_retry,lang,filter_fn,order_id,expiry_time,uploaded_images
0,jhfkv1,2024-11-15 00:07:03.881000+00:00,2024-11-15 16:34:19.855000+00:00,2024-11-15 16:39:59.941000+00:00,True,id,transformToMentari,order-jhfkv1-1731688500519,2024-11-15 23:37:01+00:00,[{'asset_id': 'aad9cf11f4c666f64b8bc8785fafdcc...
1,6hql8y,2024-11-15 00:07:03.881000+00:00,2024-11-15 15:21:46.075000+00:00,2024-11-15 15:25:43.236000+00:00,True,id,transformToMentari,order-6hql8y-1731684131757,2024-11-15 22:24:12+00:00,[{'asset_id': 'e2b8610cb7e5e1828b8a1e669c2f7b1...
2,m8o8ei,2024-11-15 00:07:03.881000+00:00,2024-11-15 15:06:24.957000+00:00,2024-11-15 15:11:40.850000+00:00,True,id,transformToMentari,order-m8o8ei-1731683227256,2024-11-15 22:09:08+00:00,[{'asset_id': '677ca00a18cd57202df0f8810851dcd...
3,e3rco1,2024-11-15 00:07:03.881000+00:00,2024-11-15 14:54:42.354000+00:00,2024-11-15 14:59:11.308000+00:00,True,id,transformToMentari,order-e3rco1-1731682500940,2024-11-15 21:57:01+00:00,[{'asset_id': '28b3015e98521d1edd9aaeef88198b4...
4,gdamqs,2024-11-15 00:07:03.881000+00:00,2024-11-15 14:31:49.048000+00:00,2024-11-15 14:35:36.542000+00:00,True,id,transformToJelaga,order-gdamqs-1731681131826,2024-11-15 21:34:12+00:00,[{'asset_id': '8a191c0e09aa78d196272dc95e2e519...


In [24]:
def create_combined_summary(joined_data):
    # Extract date from start_time for grouping
    joined_data['date'] = joined_data['start_time'].dt.date
    
    # Create marketing & sales summary
    marketing_sales_summary = joined_data.groupby('date').agg(
        transaction_amount=('transaction_amount', 'sum'),
        total_discount=('discount', 'sum'),
        total_transactions=('transaction_amount', 'count')
    ).reset_index()

    # Create user behavior facts summary
    user_behavior_facts = joined_data.groupby('date').agg(
        total_duration=('duration', 'sum'),
        avg_duration=('duration', 'mean'),
        total_copies=('copies', 'sum'),
        avg_copies=('copies', 'mean')
    ).reset_index()

    # Create operations summary
    operations_summary = joined_data.groupby('date').agg(
        total_bytes=('total_bytes', 'sum'),
        avg_bytes=('avg_bytes', 'mean'),
        total_pixels=('total_pixels', 'sum'),
        avg_pixels=('avg_pixels', 'mean')
    ).reset_index()

    # Combine all summaries into one table by merging on the 'date' column
    combined_summary = marketing_sales_summary.merge(user_behavior_facts, on='date', how='outer')
    combined_summary = combined_summary.merge(operations_summary, on='date', how='outer')

    return combined_summary

# Call the function to create the combined summary
combined_summary = create_combined_summary(joined_data)
combined_summary.head()

,date,transaction_amount,total_discount,total_transactions,total_duration,avg_duration,total_copies,avg_copies,total_bytes,avg_bytes,total_pixels,avg_pixels
0,2024-11-15,900000,0,25,7987.494,319.49976,26,1.04,162878199.0,814390.995,90626950.0,453134.75
